In [4]:
import pandas as pd 
import datetime as dt 
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [5]:

df_track=pd.read_csv('data_110000k_lignes.csv')


In [6]:
df_track.shape
    

(110158, 33)

In [7]:
df_track.duplicated().sum()

0

In [8]:
df_track.columns

Index(['Unnamed: 0', 'artist_name', 'album_name', 'album_id', 'track_name',
       'track_id', 'popularity', 'release_date', 'duration_ms', 'explicit',
       'external_urls', 'is_local', 'preview_url', 'track_number', 'genre',
       'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms.1',
       'time_signature'],
      dtype='object')

In [9]:
def prepa(df):
    df['year']=pd.to_datetime(df["release_date"],format="%Y/%m/%d").dt.year
    df=df.drop(['Unnamed: 0', 'artist_name', 'album_name', 'album_id', 'track_name',
       'track_id','release_date','time_signature',
       'external_urls', 'preview_url', 'track_number','type',"duration_ms.1",
        'id', 'uri', 'track_href', 'analysis_url','explicit',"is_local"
       ],axis=1)
    return df 

In [10]:
df_prepa=prepa(df_track)

In [11]:
df_prepa.columns

Index(['popularity', 'duration_ms', 'genre', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'year'],
      dtype='object')

In [12]:
df_prepa = df_prepa[['popularity', 'danceability', 'energy', 'key','loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness','liveness', 'valence', 'tempo','duration_ms','genre']]

In [13]:
#df_prepa.genre.value_counts(ascending=False)[0:100].index.to_list()

In [14]:
# from collections import Counter
# outre_cat=list((Counter(df_prepa.df_prepa.genre.value_counts(ascending=False).index.to_list())-Counter(df_prepa.product_category.value_counts(ascending=False).index.to_list()[0:10])).elements())
# outre_cat
# df_prepa.genre.replace(outre_cat,"other",inplace=True)

In [15]:
X=df_prepa.drop("popularity",axis=1)
y=df_prepa["popularity"]

In [16]:
X.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms', 'genre'],
      dtype='object')

In [17]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV,KFold
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=42,shuffle=True)
# X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,train_size=0.8,random_state=42,shuffle=True)

In [23]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88126 entries, 1231 to 15795
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      88126 non-null  float64
 1   energy            88126 non-null  float64
 2   key               88126 non-null  int64  
 3   loudness          88126 non-null  float64
 4   mode              88126 non-null  int64  
 5   speechiness       88126 non-null  float64
 6   acousticness      88126 non-null  float64
 7   instrumentalness  88126 non-null  float64
 8   liveness          88126 non-null  float64
 9   valence           88126 non-null  float64
 10  tempo             88126 non-null  float64
 11  duration_ms       88126 non-null  int64  
 12  genre             88126 non-null  object 
dtypes: float64(9), int64(3), object(1)
memory usage: 9.4+ MB


In [18]:
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import joblib

cols_to_norm=X.select_dtypes(exclude='object').columns.tolist()
cols_to_encoder=X.select_dtypes(include='object').columns.to_list()
xgb=XGBRegressor()
standard=StandardScaler()
preprocessor = ColumnTransformer(
    [('scaler', StandardScaler() , cols_to_norm),('encoder', OneHotEncoder() , cols_to_encoder)])

param_grid = {
'xgbregressor__learning_rate': [0.1],
'xgbregressor__max_depth': [5],
'xgbregressor__n_estimators': [600],
    # 'xgbregressor__subsample': [0.5, 0.7, 0.],
    # 'xgbregressor__colsample_bytree': [0.5,],
    # 'xgbregressor__gamma': [1],
    # 'xgbregressor__min_child_weight': [1, 3, 5]
}

# Créez votre pipeline
pipeline = Pipeline([
('columntransformer', preprocessor),

('xgbregressor', xgb)
])
experiment_name = "XGBoost"

# Créez votre objet RandomizedSearchCV avec KFold comme validateur croisé
random_search = RandomizedSearchCV(pipeline, param_grid,n_iter=50, cv=KFold(n_splits=5),verbose=5,)

# Entraînez le modèle en utilisant les données d'entraînement
random_search.fit(X_train, y_train)


joblib.dump(random_search,'random_shearch.pkl')

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.741 total time=  26.3s
[CV 2/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.735 total time=  26.4s
[CV 3/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.733 total time=  31.8s
[CV 4/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.733 total time=  25.9s
[CV 5/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.728 total time=  39.8s


['random_shearch.pkl']

In [22]:
df_prepa.genre.unique()

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro',
       'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage',
       'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge',
       'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop', 'pop-film',
       'power-pop', '

## MLFLOW

In [19]:
blabla

NameError: name 'blabla' is not defined

In [ ]:
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import mlflow

cols_to_norm=X.select_dtypes(exclude='object').columns.tolist()
cols_to_encoder=X.select_dtypes(include='object').columns.to_list()
xgb=XGBRegressor()
standard=StandardScaler()
preprocessor = ColumnTransformer(
    [('scaler', StandardScaler() , cols_to_norm),('encoder', OneHotEncoder() , cols_to_encoder)])

param_grid = {
'xgbregressor__learning_rate': [0.1],
'xgbregressor__max_depth': [5],
'xgbregressor__n_estimators': [600],
    # 'xgbregressor__subsample': [0.5, 0.7, 0.],
    # 'xgbregressor__colsample_bytree': [0.5,],
    # 'xgbregressor__gamma': [1],
    # 'xgbregressor__min_child_weight': [1, 3, 5]
}

# Créez votre pipeline
pipeline = Pipeline([
('columntransformer', preprocessor),

('xgbregressor', xgb)
])
experiment_name = "XGBoost"
if mlflow.get_experiment_by_name(experiment_name) is None:
    mlflow.create_experiment(experiment_name)

# Définir l'expérience active
mlflow.set_experiment(experiment_name)

# Démarrez une nouvelle exécution MLflow
mlflow.sklearn.autolog()
with mlflow.start_run(run_name="xgboost1"):


    # Créez votre objet RandomizedSearchCV avec KFold comme validateur croisé
    random_search = RandomizedSearchCV(pipeline, param_grid,n_iter=50, cv=KFold(n_splits=5),verbose=5,)

    # Entraînez le modèle en utilisant les données d'entraînement
    random_search.fit(X_train, y_train)



    # mlflow.log_params(random_search.best_params_)

    # # Enregistrez la performance du modèle sur les données d'entraînement
    # mlflow.log_metric("score_train", random_search.best_score_)

    # # Enregistrez la performance du modèle sur les données de test
    # mlflow.log_metric("score_test", random_search.score(X_test, y_test))

    # # Enregistrez le modèle dans un format pouvant être chargé ultérieurement pour une utilisation ou un déploiement ultérieur
    # # mlflow.sklearn.log_model(random_search.best_estimator_, artifact_path="Xgboost")
    


2023/02/28 09:44:46 WARNING mlflow.utils: Truncated the value of the key `estimator`. Truncated value: `Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['danceability', 'energy',
                                                   'key', 'loudness', 'mode',
                                                   'speechiness',
                                                   'acousticness',
                                                   'instrumentalness',
                      ...`


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.364 total time=  15.4s
[CV 2/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.365 total time=  15.0s
[CV 3/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.371 total time=  15.1s
[CV 4/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.370 total time=  15.4s
[CV 5/5] END xgbregressor__learning_rate=0.1, xgbregressor__max_depth=5, xgbregressor__n_estimators=600;, score=0.364 total time=  15.4s


2023/02/28 09:46:26 INFO mlflow.sklearn.utils: Logging the 5 best runs, no runs will be omitted.
2023/02/28 09:46:26 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('columntransformer', ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                 ['danceability', 'energy', 'key', 'loudness',
                                  'mode', 'speechiness', 'acousticness',
                                  'instrumentalness', 'liveness', 'valence',
                                  'tempo', 'duration_ms']),
                                ('encoder', OneHotEncoder(), [])])), ('xgbregressor', XGBRegressor(base_score=None, booster=...`
2023/02/28 09:46:26 WARNING mlflow.utils: Truncated the value of the key `xgbregressor`. Truncated value: `XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
          

In [ ]:
# Set the experiment name and retrieve its ID
experiment_name = "XGBoost"
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
run = mlflow.search_runs(experiment_ids=experiment_id, order_by=["start_time DESC"], max_results=1).iloc[0]
run_id = run.run_id
run

run_id                                      0bb30d64f42d4deaa768dc4d3197db0c
experiment_id                                             360426153041130070
status                                                              FINISHED
artifact_uri               file:///home/apprenant/Bureau/Projet-r-capitul...
start_time                                  2023-02-28 08:44:46.223000+00:00
                                                 ...                        
tags.mlflow.user                                                   apprenant
tags.mlflow.runName                                            aged-frog-274
tags.mlflow.source.type                                                LOCAL
tags.mlflow.parentRunId                     ba111d0e1d9b4d49b352b08d1f425374
tags.mlflow.autologging                                              sklearn
Name: 0, Length: 84, dtype: object

In [ ]:
test = pd.DataFrame(X.iloc[0,:]).transpose()
test

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.676,0.461,1.0,-6.746,0.0,0.143,0.0322,0.000001,0.358,0.715,87.917,230666.0


In [ ]:
import mlflow
X_test=df_prepa.iloc[1,:]
# Rechercher la course en fonction du nom de la course et du nom de l'exécution
experiment_name = "XGBoost"
run_name = "xgboost1"
runs = mlflow.search_runs(experiment_ids=mlflow.get_experiment_by_name(experiment_name).experiment_id, 
                          filter_string=f"tags.mlflow.runName = '{run_name}'")
# Récupérer l'ID de la course
run_id = runs.iloc[0].run_id

# Charger le modèle enregistré
loaded_model = mlflow.sklearn.load_model(f"runs:/{run_id}/model")

# Utiliser le modèle pour faire des prédictions
predictions = loaded_model.predict(test)
predictions

array([47.87017], dtype=float32)

#

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
# Définissez les paramètres à tester dans GridSearchCV
grid_params = {
    'polynomialfeatures__degree': [2],
    'randomforestregressor__n_estimators': [1000],
    'randomforestregressor__max_depth': [9],
}

# Créez le pipeline
pipeline = Pipeline([
    ('polynomialfeatures', PolynomialFeatures()),
    ('randomforestregressor', RandomForestRegressor())
])
kf=KFold(n_splits=5,random_state=42,shuffle=True)
# Créez l'objet GridSearchCV
grid_search = RandomizedSearchCV(pipeline, grid_params, cv=kf,verbose=5 )

# Entraînez le modèle en utilisant les données d'entraînement
grid_search.fit(X_train, y_train)

# Affichez les meilleurs paramètres trouvés par GridSearchCV
print(grid_search.best_params_)
print(grid_search.score(X_train, y_train))
print(grid_search.score(X_test, y_test))
# Prédisez sur les données de test en utilisant le modèle entraîné avec les meilleurs paramètres
with mlflow.start_run():

    # Enregistrez les paramètres
    mlflow.log_params(random_search.best_params_)

    # Enregistrez la performance du modèle sur les données d'entraînement
    mlflow.log_metric("score_train", random_search.best_score_)

    # Enregistrez la performance du modèle sur les données de test
    mlflow.log_metric("score_test", random_search.score(X_test, y_test))

    # Enregistrez le modèle dans un format pouvant être chargé ultérieurement pour une utilisation ou un déploiement ultérieur
    mlflow.sklearn.log_model(random_search.best_estimator_, "xgboost")




2023/02/28 09:46:26 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '4666078cb2b44ad58c7c1c75900cf352', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Fitting 5 folds for each of 1 candidates, totalling 5 fits


KeyboardInterrupt: 